# Hackathon Dotz

##### Classificar as observações por (SUB-CATEGORIA | CATEGORIA | DEPARTAMENTO)

In [0]:
# Importação dos pacotes para Analise
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter(action = 'ignore')

In [0]:
# Importação dos dataset
df = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

### Etapa de Preparação do Dados

In [0]:
# Variáveis importantes
pontuacao = ['.',',','-','+',':',';','&','+','/','!','#','%','(',')','  ']
# Palavras para retirar da análise
stop_words = ['em','sao','ao','de','da','do','para','c','kg','un','ml','pct','und','das','no','ou', 
              'com','sem','gfa','jg','la','1','2','3','4','5','6','7','8','9','0','a','b','c','d','e',
              'f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','x','w','y','z']
# Tamanho da validação de teste
test_size = 0.01
random_state = 1
# Parametros do modelo LinearSVC
penalty = 'l1'
dual = False
tol = 0.001
max_iter = 4000
loss = 'hinge'
multi_class = 'crammer_singer'
# Parametros do vetor CountVectorizer
ngram_range = (1, 2)
strip_accents = 'ascii'
# Parametros do vetor TfidfTransformer
use_idf = True

In [11]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)
# Dados de Teste
df2['DESC_AJUSTADA'] = df2['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
for x in pontuacao:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, ' ')
  # Dados de Teste
  df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].str.replace(x, ' ')
  
df.head(2)

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESC_AJUSTADA
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE,PASTA INT VITAPOWER KG AMEND SHOT
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS C


### Função Treinamento

In [0]:
# Função Treinamento, Teste, Resultado
def train(feature, target, new_feature, new_target):
  cvt = CountVectorizer(ngram_range=ngram_range, strip_accents=strip_accents, stop_words=stop_words)
  tfi = TfidfTransformer(use_idf=use_idf)
  clf = LinearSVC(penalty=penalty, dual=dual, multi_class=multi_class, tol=tol, loss=loss, max_iter=max_iter)

  # Criando pipeline
  clf = Pipeline([('cvt', cvt), ('tfi', tfi), ('clf', clf)])
  
  # Dividindo dataset em treino e teste
  x_train, x_test, y_train, y_test = train_test_split(feature, target, test_size=test_size, random_state=random_state)

  # Executando pipeline
  clf.fit(x_train, y_train)

  # Avaliando a performance com predição
  predicted = clf.predict(x_test)
  print('\n###Indicadores Classificação###\n')
  print(metrics.classification_report(y_test, predicted))

  #Previsão dos novos dados
  predicted_new = clf.predict(new_feature)

  # Adicionando a coluna para o novo DF
  df2[new_target] = predicted_new

  return df2

### Etapa 1 - Classificando DEPARTAMENTO

In [13]:
# Selecionando apenas o item a ser classificado e o target do DF principal
feature = df['DESC_AJUSTADA']
target = df['DEPARTAMENTO']
# Dados de Teste
new_feature = df2['DESC_AJUSTADA']
new_target = 'DEPARTAMENTO'
# Chamando Função
df2 = train(feature, target, new_feature, new_target)
df2.head()


###Indicadores Classificação###

                          precision    recall  f1-score   support

              AUTOMOTIVO       1.00      1.00      1.00         4
                 AÇOUGUE       0.83      1.00      0.91         5
      BEBIDAS ALCOÓLICAS       1.00      1.00      1.00        16
  BEBIDAS NÃO ALCOÓLICAS       1.00      0.91      0.95        11
               BRINQUEDO       0.00      0.00      0.00         1
      CAMA, MESA E BANHO       1.00      1.00      1.00         4
        CONGELADOS DOCES       1.00      1.00      1.00         4
     CONGELADOS SALGADOS       1.00      1.00      1.00         1
    CUIDADOS COM A BARBA       1.00      1.00      1.00         1
     CUIDADOS COM A CASA       0.80      1.00      0.89         4
  CUIDADOS COM A COZINHA       1.00      1.00      1.00         1
    CUIDADOS COM A ROUPA       1.00      1.00      1.00         2
 CUIDADOS COM MÃOS E PÉS       1.00      1.00      1.00         3
 CUIDADOS COM O BANHEIRO       1.00      

,0,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO
0,1,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO G,MERCEARIA SALGADA
1,2,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C DR PR BC R,DECORAÇÃO
2,3,MIST PREP LIQ CHA VDE LINEA 56ML-CX LIMAO,MIST PREP LIQ CHA VDE LINEA ML CX LIMAO,MERCEARIA DOCE
3,4,ALICATE CUT+ESPAT MUNDIAL SANDY AMARELO,ALICATE CUT ESPAT MUNDIAL SANDY AMARELO,CUIDADOS COM MÃOS E PÉS
4,5,MEIA PIZZA PEPPERONI UN,MEIA PIZZA PEPPERONI UN,PRATOS PRONTOS


### Etapa 2 - Classificando CATEGORIA

In [14]:
# Selecionando apenas o item a ser classificado e o target do DF principal
feature = df['DEPARTAMENTO'] + ' ' + df['DESC_AJUSTADA']
target = df['CATEGORIA']
# Dados de Teste
new_feature = df2['DEPARTAMENTO'] + ' ' + df2['DESC_AJUSTADA']
new_target = 'CATEGORIA'
# Chamando Função
df2 = train(feature, target, new_feature, new_target)
df2.head()


###Indicadores Classificação###

                                   precision    recall  f1-score   support

              ACESSÓRIOS MÃO E PÉ       1.00      1.00      1.00         1
           ACESSÓRIOS PARA CABELO       1.00      1.00      1.00         1
                         ADOÇANTE       1.00      1.00      1.00         1
               AGUARDENTE/CACHAÇA       1.00      1.00      1.00         2
                          ALGODÃO       1.00      1.00      1.00         1
               ALIMENTO PARA AVES       1.00      1.00      1.00         1
               ALIMENTO PARA CÃES       1.00      1.00      1.00         3
                        ALVEJANTE       1.00      1.00      1.00         1
                APARELHO DE BARBA       1.00      1.00      1.00         1
                            ARROZ       1.00      1.00      1.00         1
              ARTIGOS PARA FESTAS       1.00      1.00      1.00         4
                        ASPIRADOR       1.00      1.00      1.00 

,0,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,CATEGORIA
0,1,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO G,MERCEARIA SALGADA,SAL
1,2,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C DR PR BC R,DECORAÇÃO,ENFEITES DE NATAL
2,3,MIST PREP LIQ CHA VDE LINEA 56ML-CX LIMAO,MIST PREP LIQ CHA VDE LINEA ML CX LIMAO,MERCEARIA DOCE,FARINÁCEO
3,4,ALICATE CUT+ESPAT MUNDIAL SANDY AMARELO,ALICATE CUT ESPAT MUNDIAL SANDY AMARELO,CUIDADOS COM MÃOS E PÉS,ACESSÓRIOS MÃO E PÉ
4,5,MEIA PIZZA PEPPERONI UN,MEIA PIZZA PEPPERONI UN,PRATOS PRONTOS,PIZZA


### Etapa 3 - Classificando SUB-CATEGORIA

In [15]:
# Selecionando apenas o item a ser classificado e o target do DF principal
feature = df['CATEGORIA'] + ' ' + df['DESC_AJUSTADA']
target = df['SUB-CATEGORIA']
# Dados de Teste
new_feature = df2['CATEGORIA'] + ' ' + df2['DESC_AJUSTADA']
new_target = 'SUB-CATEGORIA'
# Chamando Função
df2 = train(feature, target, new_feature, new_target)
df2.head(20)


###Indicadores Classificação###

                                precision    recall  f1-score   support

                       ABÓBORA       0.50      1.00      0.67         1
                       AEROSOL       0.00      0.00      0.00         1
           AEROSOL COM PERFUME       1.00      1.00      1.00         1
                        ALFACE       1.00      1.00      1.00         1
             ALMOFADA EXPRESSO       0.00      0.00      0.00         1
          ALMOFADA EXTRA FORTE       0.00      0.00      0.00         1
          ALMOFADA TRADICIONAL       0.25      1.00      0.40         1
                      ALVEJADO       1.00      1.00      1.00         1
                      AMENDOIM       1.00      1.00      1.00         1
               ANTI-BACTERIANO       0.00      0.00      0.00         1
                    ANTI-FRIZZ       0.00      0.00      0.00         1
                  ANTISSÉPTICO       1.00      1.00      1.00         1
                      APARELH

,0,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,CATEGORIA,SUB-CATEGORIA
0,1,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO G,MERCEARIA SALGADA,SAL,SAL DO HIMALAIA
1,2,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C DR PR BC R,DECORAÇÃO,ENFEITES DE NATAL,BOLA DE NATAL
2,3,MIST PREP LIQ CHA VDE LINEA 56ML-CX LIMAO,MIST PREP LIQ CHA VDE LINEA ML CX LIMAO,MERCEARIA DOCE,FARINÁCEO,MISTURAS
3,4,ALICATE CUT+ESPAT MUNDIAL SANDY AMARELO,ALICATE CUT ESPAT MUNDIAL SANDY AMARELO,CUIDADOS COM MÃOS E PÉS,ACESSÓRIOS MÃO E PÉ,ALICATE
4,5,MEIA PIZZA PEPPERONI UN,MEIA PIZZA PEPPERONI UN,PRATOS PRONTOS,PIZZA,PRESUNTO
5,6,JARRA ACASA VIDRO CURVES OJG378,JARRA ACASA VIDRO CURVES OJG,UTILIDADES DE COZINHA,UTENSÍLIOS DE VIDRO,JARRA/GARRAFA
6,7,FILME DE PVC CONSERV PACK 30M,FILME DE PVC CONSERV PACK M,DESCARTÁVEIS,FILME PVC,FILME PVC
7,8,CESTO VIME CRAFTWORK S-25X15XH13,CESTO VIME CRAFTWORK S XXH,LAVANDERIA,ORGANIZADOR,CESTO DE ROUPA
8,9,MAC BORTOLINI 500GR OVOS ESPAGUETE .,MAC BORTOLINI GR OVOS ESPAGUETE,MERCEARIA SALGADA,MASSA SECA,ESPAGUETE
9,10,COND NOVEX OLEO OJON 300ML,COND NOVEX OLEO OJON ML,CUIDADOS COM O CABELO,CONDICIONADOR,TRADICIONAL


In [0]:
# Exportando classificação
df_result = df2[['DESCRIÇÃO PARCEIRO', 'SUB-CATEGORIA', 'CATEGORIA']]
df_result.to_csv('result.csv')